# Load Shedding, Shifting and Elastic Demand

This example is a simple illustration that aims to explain the use of load shedding, load shifting and elastic demand in PyPSA. The example is designed to simulate a single day of electrical consumption in a small network with solar generation, which could represent a single house or a small group of houses.

In [ ]:
import pypsa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Build example network

First, let's start building a minimal example network for which we demonstrate the demand modelling cases.
The example network is represented as single bus: 
- with one PV generator
- and one baseload electricity demand profile
- between 04:00 - 20:00

In [ ]:
network = pypsa.Network()

# Add a bus
network.add("Bus", "My bus")

# Add fuel types
network.add("Carrier", "solar")
network.add("Carrier", "load")

# Add snapshots to define considered time span
network.set_snapshots(pd.date_range("2023-01-01 04:00", "2023-01-01 20:00", freq="H"))

# Add solar generation
pv_pu = [
    0.0,
    0.0,
    0.0,
    0.2,
    0.4,
    0.65,
    0.85,
    0.9,
    0.85,
    0.65,
    0.4,
    0.3,
    0.2,
    0.1,
    0.0,
    0.0,
    0.0,
]

# Add solar generation
network.add(
    "Generator",
    "solar",
    bus="My bus",
    p_nom=0.4,
    carrier="solar",
    p_max_pu=pv_pu,
    marginal_cost=0.01,
)

# Add baseload
load = pd.Series(0.2, index=range(17))  # constant baseload
network.add("Load", "baseload", bus="My bus", p_set=load.values)

Plotting the minimal example network data, we can see that supply **cannot** equal demand at all time steps. This means that a stable grid operation is not possible.

In [ ]:
fig, ax = plt.subplots()

(network.generators_t.p_max_pu * network.generators.p_nom).plot(ax=ax)
network.loads_t.p_set.plot(ax=ax)

ax.set_ylabel("Power (MW)")

Trying to optimize this network without any possibility of achieving the energy balance at every time step will lead to a infeasible optimization.

In [ ]:
network.optimize()

## Load shedding

"Load shedding" in power networks often happens under extreme conditions, e.g. to prevent network faults. In models, we consider load shedding as extremely expensive and unconstrained generators that can practically decrease the demand to zero. Tip: Load shedding generators are often used to track-down infeasibilities from power shortages (as above) or too tight constraints.

In [ ]:
# Add load shedding as generator
network.add(
    "Generator",
    "load shedding",
    bus="My bus",
    p_nom=50,
    carrier="load",
    p_max_pu=1.0,
    marginal_cost=1e2,  # 100 €/MWh
    p_nom_extendable=True,
)

After adding load shedding, the optimization will pass as the nodal energy balance is fulfilled

In [ ]:
network.optimize()

Looking at the optimization results, the mismatch is exactly compensated by load shedding.

In [ ]:
fig, ax = plt.subplots()

network.generators_t.p.plot(ax=ax)
network.loads_t.p.plot(ax=ax)

ax.set_ylabel("Power (MW)")

## Load shifting

"Load shifting" is often used to provide network benefits arising from demand side flexibility. One example application are smart thermostats that can shift the timing of energy use by automatically shifting the **electric** heating or cooling demand from one to another period e.g. from peak to off-peak period. We model load shifting as storage units, which are often described as energy time-shifters.

In [ ]:
# add battery as carrier
network.add("Carrier", "thermostat")

# add smart thermostat as storage unit
network.add(
    "StorageUnit",
    "smart thermostat",
    bus="My bus",
    p_nom=0,
    carrier="heat",
    p_nom_extendable=True,
    standing_loss=0.01,  # penalize if load shifting is not applied at the first instance of mismatch
    cyclic_state_of_charge=True,
    max_hours=6,
)

### Let's solve and plot load shifting and shedding 

In [ ]:
network.optimize()

In [ ]:
fig, ax = plt.subplots()

network.generators_t.p.plot(ax=ax)
network.storage_units_t.p.plot(ax=ax)
network.loads_t.p.plot(ax=ax)

ax.set_ylabel("Power (MW)")

We can observe from the least cost solution the following:
- The smart thermostat is increasing the demand during the peak solar generation, to save energy when there is a lack of supply
- In the first and last hours of the day, load shedding is applied since there is not enough storage or generation available

## Elastic Demand

Demand is slightly "elastic", meaning consumers would use less energy if its too expensive, or would use more energy if its low costs. Think about yourself. Would you consume less energy if it cost 5x more than today? The elasticity of demand is usually nonlinear and highly depended on individuals. If you are interested to read more into this, [Labandeira et al. (2017)](https://doi.org/10.1016/j.enpol.2017.01.002) surveys 428 papers that use various econometric techniques to evaluate energy demand elasticity to price.

We model eleastic demand by:
  - using a linear elasticity curve (price & demand relation)
  - iteration model runs with updated demands
  
Note. Demand is model input & the electricity price is the model output. See [here](https://github.com/PyPSA/PyPSA/issues/574) a nice illustration. The below elasticity curve is made up for demonstration purposes.

In [ ]:
# assuming a linear demand response to price
def elasticity(system_cost_per_MWh):
    # y = mx + t
    # x = y - t / m
    return (system_cost_per_MWh - 120) / -300

In [ ]:
# Define a range of input values
x = np.arange(0, 300, 10)

# Calculate output values using the elasticity function
y = [elasticity(i) for i in x]

# Create the plot
plt.plot(y, x)
plt.xlabel("Demand [MWh]")
plt.ylabel("Electricity price [Eur/MWh]")
plt.title("Random elasticity curve for the example")
plt.xlim([0, 0.5])
plt.ylim([0, 150])
plt.show()

Using the same network as in the previous sections, we perform the iteration:
- add demand to network
- optimize network to get the system price
- update the demand by adding the system price to the elasticity curve

In [ ]:
base_load = 0.2
all_networks = []

for _ in range(6):
    network = network.copy()
    load = pd.Series(
        base_load, index=range(17)
    )  # variable baseload based on base_load_values
    network.remove("Load", "baseload")
    network.add("Load", "baseload", bus="My bus", p_set=load.values)
    network.lopf()

    # Estimated price of the network in euros per MWh
    system_cost_per_MWh = (
        network.objective + network.objective_constant
    ) / network.loads_t.p.sum().sum()

    # assuming a linear demand response to price
    new_load = elasticity(system_cost_per_MWh)
    base_load = new_load

    all_networks.append(network)

### Let's plot the elastic demand

In [ ]:
### Demand and price values from the created networks
average_loads = []
prices = []
for n in all_networks:
    average_loads.append(n.loads_t.p.mean().sum())
    prices.append((n.objective + n.objective_constant) / n.loads_t.p.sum().sum())

In [ ]:
first_demand_iteration = average_loads[0]
updated_average_loads = average_loads[
    1:
]  # demand value in first iteration didn't have a price
updated_prices = prices[:-1]  # last price value wasn't used

# Scatter plot of the points
plt.scatter(updated_average_loads, updated_prices, label="Elastic demand")

# Add numbers to each point
for i, (x, y) in enumerate(zip(updated_average_loads, updated_prices)):
    plt.annotate(
        "Iteration " + str(i + 1),
        (x, y),
        textcoords="offset points",
        xytext=(30, 5),
        ha="center",
    )

# Fit a linear curve to the data
slope, intercept = np.polyfit(updated_average_loads, updated_prices, 1)

# Linear curve
x = np.linspace(min(updated_average_loads), max(updated_average_loads), 100)
y = slope * x + intercept
plt.plot(x, y, label=f"Linear curve", color="blue")

plt.xlabel("Demand (MWh)")
plt.ylabel("Price (Euro/MWh)")
plt.title("Elasticity Curve")
plt.legend(loc="upper right")
plt.xlim([0.15, 0.35])
plt.ylim([35, 60])
plt.plot(x, y)
plt.show()

We can observe from the elastic demand the following:
- Our initial average demand estimate of 0.2 MWh lead to a price resulting in more demand -> **iteration 1**.
- Using the new demand derieved from the elasticity curve and **iteration 1**, increased the average system price -> **iteration 2**
- ...
Luckily, after several iterations we observe that the results are **converging.**

**Note.** Here we solve the elastic demand as linear programming (LP) problem and several iterations. It is also possible to solve the problem with **one single** optimization run. However, this requires to reformulate the problem as quadratic programming (QP) problem, see [here](https://github.com/PyPSA/PyPSA/issues/574#issuecomment-1451865672). PyPSA can also do that, however, we need to add the QP solving capability to Linopy. This is already tracked as [feature request](https://github.com/PyPSA/linopy/issues/18). Stay tuned!